In [21]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle

alpha=2.7
beta=3.9

In [22]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict(NN, x):
    activations = x.reshape((-1, ))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    logit = 1/(1+jnp.exp(-logit))
    return logit.squeeze()

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = vmap(gpredict, in_axes=(None, 0))

In [23]:
def pi(x):
    from scipy.special import gamma
    B_ab = (gamma(alpha) * gamma(beta)) / gamma(alpha + beta)
    pdf = (x**(alpha - 1)) * ((1 - x)**(beta - 1)) / B_ab
    return pdf
vpi = vmap(pi, in_axes=(0,))

def pi_pre(NN, x):
    result = gpredict(NN, x)
    return result

@jit
def obj(NN, x_batch):
    def obj1(NN, x):
        return jnp.log(pi(x)) - jnp.log(pi_pre(NN, x))
    vobj1 = vmap(obj1, in_axes=(None, 0))

    result = vobj1(NN, x_batch)
    result = jnp.square(result)
    result = jnp.mean(result)
    return result
jgo = jit(grad(obj))

@jit
def evaluation(NN, test_zu):
    z_batch, u_batch = test_zu[:, 0], test_zu[:, 1]
    result = jnp.square(vpredict(NN, z_batch) - u_batch)
    result = jnp.mean(result)
    return result

In [ ]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
test_zu = np.load(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (TM)/test_zu(a{alpha}b{beta}).npy')
lowest = evaluation(NN, test_zu)
Lloss = []
Llowest = []
NN_best = NN
for epoch in range(2000000):
    x_batch = np.random.beta(alpha, beta, (10000, 1))

    grads = jgo(NN, x_batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, x_batch).item())
    Llowest.append(lowest.item())
    print(f"Epoch: {epoch}, loss:{obj(NN, x_batch)}, {lowest}")

    if  evaluation(NN, test_zu)<lowest:
        lowest = evaluation(NN, test_zu)
        NN_best = NN

    if epoch % 10000 == 0:
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (TM)/a{alpha}b{beta}/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (TM)/a{alpha}b{beta}/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (TM)/a{alpha}b{beta}/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (TM)/a{alpha}b{beta}/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

Streaming output truncated to the last 5000 lines.
Epoch: 1995000, loss:1.2963485573891376e-07, 1.6236484379206217e-11
Epoch: 1995001, loss:2.181705980319748e-07, 1.6236484379206217e-11
Epoch: 1995002, loss:1.4085743771374837e-07, 1.6236484379206217e-11
Epoch: 1995003, loss:2.1068643718535895e-07, 1.6236484379206217e-11
Epoch: 1995004, loss:1.7580686062501627e-07, 1.6236484379206217e-11
Epoch: 1995005, loss:1.8945754618471256e-06, 1.6236484379206217e-11
Epoch: 1995006, loss:2.737236002303689e-07, 1.6236484379206217e-11
Epoch: 1995007, loss:7.25743575458182e-07, 1.6236484379206217e-11
Epoch: 1995008, loss:1.3044876823187224e-06, 1.6236484379206217e-11
Epoch: 1995009, loss:3.589768198253296e-07, 1.6236484379206217e-11
Epoch: 1995010, loss:1.6172367622857564e-07, 1.6236484379206217e-11
Epoch: 1995011, loss:2.0498856656558928e-07, 1.6236484379206217e-11
Epoch: 1995012, loss:2.5348080612275226e-07, 1.6236484379206217e-11
Epoch: 1995013, loss:1.1732462468216909e-07, 1.6236484379206217e-11
Ep